In [ ]:
!pip install neo4j transformers accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:

from huggingface_hub import notebook_login
notebook_login()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import pandas as pd
import os
from peft import PeftModel, PeftConfig
import torch
from google.colab import files
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files


Mounted at /content/drive


In [ ]:
# load sample questions
# Get the current script directory
current_path = os.getcwd()
print("Current Path:", current_path)

# Load questions CSV using the correct path
csv_path = os.path.join(current_path, 'drive/MyDrive/cs696project/text2cypher_gemini.csv')
print("CSV Path:", csv_path)
# Check if file exists before reading
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(df.head())
else:
    print(f"Error: File not found - {csv_path}")

Current Path: /content
CSV Path: /content/drive/MyDrive/cs696project/text2cypher_gemini.csv
                                            question         database  \
0  Which movies from France have an IMDb rating h...  recommendations   
1  What are the titles of the top 5 highest-gross...  recommendations   
2  List the first 3 movies directed by Harold Lloyd.  recommendations   
3  What genres are associated with the movie "Toy...  recommendations   
4  What is the average IMDb rating of movies rele...  recommendations   

                                              output  \
0  <explanation>\n1. We start by finding movies w...   
1  <explanation>\nThis Cypher query returns the t...   
2  <explanation>\n1. We need to find the director...   
3  <explanation>\nThis query starts by finding th...   
4  <explanation>\n1. **Find movies released in 19...   

                       type  \
0  Simple Retrieval Queries   
1  Simple Retrieval Queries   
2  Simple Retrieval Queries   
3  Simpl

In [ ]:
!nvidia-smi

In [ ]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://demo.neo4jlabs.com:7687"
AUTH = ("recommendations", "recommendations")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    print("Connection established.")

Connection established.


In [ ]:
instruction = (
    "Generate Cypher statement to query a graph database. "
    "Use only the provided relationship types and properties in the schema. \n"
    "Schema: {schema} \n Question: {question}  \n Cypher output: "
)

def prepare_chat_prompt(question, schema) -> list[dict]:
    chat = [
        {
            "role": "user",
            "content": instruction.format(
                schema=schema, question=question
            ),
        }
    ]
    return chat

def _postprocess_output_cypher(output_cypher: str) -> str:
    # Remove any explanation. E.g.  MATCH...\n\n**Explanation:**\n\n -> MATCH...
    # Remove cypher indicator. E.g.```cypher\nMATCH...```` --> MATCH...
    # Note: Possible to have both:
    #   E.g. ```cypher\nMATCH...````\n\n**Explanation:**\n\n --> MATCH...
    partition_by = "**Explanation:**"
    output_cypher, _, _ = output_cypher.partition(partition_by)
    output_cypher = output_cypher.strip("`\n")
    output_cypher = output_cypher.lstrip("cypher\n")
    output_cypher = output_cypher.strip("`\n ")
    return output_cypher

In [ ]:
model_name = "neo4j/text2cypher-gemma-2-9b-it-finetuned-2024v1"
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    attn_implementation="eager",
    low_cpu_mem_usage=True,
).to(device="cuda:0", non_blocking=True)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# sample question
question = "What are the movies of Tom Hanks?"
# schema = "(:Actor)-[:ActedIn]->(:Movie)" # Check the NOTE below on creating your own schemas
schema = ""
new_message = prepare_chat_prompt(question=question, schema=schema)
prompt = tokenizer.apply_chat_template(new_message, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(prompt, return_tensors="pt", padding=True)

# Any other parameters
model_generate_parameters = {
    "top_p": 0.9,
    "temperature": 0.1,
    "max_new_tokens": 512,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

inputs.to(model.device)
model.eval()
with torch.no_grad():
    tokens = model.generate(**inputs, **model_generate_parameters)
    tokens = tokens[:, inputs.input_ids.shape[1] :]
    raw_outputs = tokenizer.batch_decode(tokens, skip_special_tokens=True)
    outputs = [_postprocess_output_cypher(output) for output in raw_outputs]

print(outputs)

['MATCH (a:Actor {Name: "Tom Hanks"})-[:ACTED_IN]->(m:Movie) RETURN m.Title AS MovieTitle']


In [ ]:
rec_df = df[(df['database'] == 'recommendations') & (df['returns_results'] == True)]
sample = 20
# Randomly sample 10 elements (if available)
sampled_df = rec_df.sample(n=sample, random_state=42)


In [ ]:

model_generate_parameters = {
    "top_p": 0.9,
    "temperature": 0.1,
    "max_new_tokens": 512,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

cypher_outputs = []  # generated cypher output
execution_errors = []  # executable?

for i in range(sample):
    question = sampled_df.iloc[i]['question']
    print(question)
    new_message = prepare_chat_prompt(question=question, schema='')
    prompt = tokenizer.apply_chat_template(new_message, add_generation_prompt=True, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    inputs.to(model.device)
    model.eval()
    with torch.no_grad():
        tokens = model.generate(**inputs, **model_generate_parameters)
        tokens = tokens[:, inputs.input_ids.shape[1] :]
        raw_outputs = tokenizer.batch_decode(tokens, skip_special_tokens=True)
        output = [_postprocess_output_cypher(output) for output in raw_outputs]
        cypher_outputs.append(output[0])  # record cypher output
        ###########################  execute it on the database
        try:
            records, summary, keys = driver.execute_query(
            cypher_outputs[i],  # Fixed variable name `g.name` → `p.name`
            database_="recommendations",
            )

            # # Loop through results and print as dictionary
            # for record in records:
            #     print(record.data())

            # # Summary information
            # print("The query `{query}` returned {records_count} records in {time} ms.".format(
            #     query=summary.query, records_count=len(records),
            #     time=summary.result_available_after
            # ))
            execution_errors.append(False)
        except Exception as e:
            print(f"An error occurred: {e}")
            execution_errors.append(True)
            continue  # Skip to the next iteration if an error occurs
      ###########################
print(cypher_outputs)

What is the average IMDB rating of movies based on books?


<ipython-input-63-0201bd9e418b>:28: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


An error occurred: {code: Neo.ClientError.Statement.SyntaxError} {message: Variable `rating` not defined (line 1, column 73 (offset: 72))
"MATCH (m:Movie)-[:BASED_ON]->(b:Book) WITH m.IMDB_RATING AS rating, avg(rating) AS average_rating RETURN average_rating"
                                                                         ^}
List the movies released in the same year as "Boxtrolls, The".


List the actors who have acted in both 'Comedy' and 'Drama' movies.


Which director has the highest ratio of box office revenue to budget?
An error occurred: {code: Neo.ClientError.Statement.ArithmeticError} {message: / by zero}
List the actors who have acted in both "Boxtrolls, The" and "Open Season".


...

What is the average IMDb rating of movies released in the 21st century?
List the names of all actors who have won both an Oscar and a Golden Globe award.


What is the average rating given by users to movies in the "Adventure" genre?
What is the total revenue of movies released by Warner Bros. Pictures?


What is the most common language among movies with a runtime longer than 3 hours?


List all the movies that have been adapted into TV shows.


List all movies that have a budget greater than 200 million and a runtime less than 2 hours.
['MATCH (m:Movie)-[:BASED_ON]->(b:Book) WITH m.IMDB_RATING AS rating, avg(rating) AS average_rating RETURN average_rating', "MATCH (m1:Movie)-[:RELEASED_IN]->(y1:Year)<-[:RELEASED_IN]-(m2:Movie) WHERE m1.title = 'Boxtrolls, The' RETURN m2.title", "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) WHERE m.Genre IN ['Comedy', 'Drama'] RETURN DISTINCT a.Name", 'MATCH (d:Director) -[r:DIRECTED]->(m:Movie) WITH d, sum(r.BoxOfficeRevenue) AS totalRevenue, sum(r.Budget) AS totalBudget RETURN d.Name AS Director, totalRevenue / totalBudget AS RevenueRatio ORDER BY RevenueRatio DESC LIMIT 1', "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) WHERE m.Title IN ['Boxtrolls, The', 'Open Season'] RETURN a.Name AS ActorName, m.Title AS MovieTitle", 'MATCH (u:User {name: "Omar Huffman"})-[:HAS_BIRTH_YEAR]->(b:BirthYear) MATCH (m:Movie)-[:RELEASED_IN]->(b) RETURN m.title AS MovieTitle, b.year AS BirthYear', "MATCH (m:Movie) WHERE m.

In [ ]:
sampled_df['finetuned_results'] = cypher_outputs
sampled_df['execution_errors'] = execution_errors
sampled_df['execution_errors'].value_counts()
# csv_filename = "10_Samples.csv"
# sampled_df.to_csv(csv_filename, index=False)  # Save without the index

# # Download the file
# files.download(csv_filename)

,count
execution_errors,
False,17
True,3


In [ ]:
# neo4j official
current_path = os.getcwd()
print("Current Path:", current_path)
# splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet(path=f'{current_path}/drive/MyDrive/cs696project/train-00000-of-00001.parquet', engine="pyarrow")
df.head()

,question,schema,cypher,data_source,instance_id,database_reference_alias
0,Which 3 countries have the most entities linke...,Node properties:\n- **Country**\n - `location...,MATCH (f:Filing)-[:BENEFITS]->(e:Entity)-[:COU...,neo4jLabs_synthetic_gpt4o,instance_id_41185,neo4jlabs_demo_db_fincen
1,What are the names of the first 3 organization...,Node properties:\n- **Person**\n - `name`: ST...,MATCH (o:Organization)-[:HAS_CEO]->(ceo:Person...,neo4jLabs_synthetic_gpt4turbo,instance_id_26598,neo4jlabs_demo_db_companies
2,List the names of the games played by streams ...,Node properties:\n- **Stream**\n - `createdAt...,MATCH (s:Stream)-[:PLAYS]->(g:Game) WHERE s.to...,neo4jLabs_synthetic_gemini,instance_id_34035,neo4jlabs_demo_db_twitch
3,For each Article find its abstract and the cou...,Graph schema: Relevant node labels and their p...,MATCH (n:Article) -[:HAS_KEY]->(m:Keyword) WIT...,neo4jLabs_functional_cypher,instance_id_3914,None
4,Find the Author for which first_name starts wi...,Graph schema: Relevant node labels and their p...,MATCH (n:Author) WHERE n.first_name STARTS WIT...,neo4jLabs_functional_cypher,instance_id_14645,None


In [ ]:

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
).to(device='cuda')
print(model.device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 130.00 MiB. GPU 0 has a total capacity of 22.16 GiB of which 45.38 MiB is free. Process 3373 has 22.11 GiB memory in use. Of the allocated memory 21.74 GiB is allocated by PyTorch, and 147.75 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
cc = 1
df_toy = df.head(10)
print(df_toy)
responses = []
# for i in range(cc):
tokenizer.pad_token = tokenizer.eos_token
question = df_toy.iloc[0]['question']
schema = df_toy.iloc[0]['schema']
print(question)
print(schema)
new_message = prepare_chat_prompt(question=question, schema=schema)
prompt = f"<|im_start|>{new_message}.<|im_end|>\n<|im_start|>assistant\n"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=1000,
    temperature=0.2,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)
response = tokenizer.decode(outputs[0], skip_special_tokens=False)
responses.append(response)
print(i)
print(responses)

In [ ]:
df_toy.iloc[0]['cypher']
df_toy.iloc[0]['question']

'Which 3 countries have the most entities linked as beneficiaries in filings?'

In [ ]:
<｜begin▁of▁sentence｜><|im_start|>[{'role': 'user', 'content': 'Generate Cypher statement to query a graph database. Use only the provided relationship types and properties in the schema. \nSchema: Node properties:\n- **Country**\n  - `location`: POINT \n  - `code`: STRING Example: "AFG"\n  - `name`: STRING Example: "Afghanistan"\n  - `tld`: STRING Example: "AF"\n- **Filing**\n  - `begin`: DATE_TIME Min: 2000-02-08T00:00:00Z, Max: 2017-09-05T00:00:00Z\n  - `end`: DATE_TIME Min: 2000-02-08T00:00:00Z, Max: 2017-11-03T00:00:00Z\n  - `originator_bank_id`: STRING Example: "cimb-bank-berhad"\n  - `sar_id`: STRING Example: "3297"\n  - `beneficiary_bank`: STRING Example: "Barclays Bank Plc"\n  - `filer_org_name_id`: STRING Example: "the-bank-of-new-york-mellon-corp"\n  - `originator_bank_country`: STRING Example: "Singapore"\n  - `beneficiary_bank_country`: STRING Example: "United Kingdom"\n  - `filer_org_name`: STRING Example: "The Bank of New York Mellon Corp."\n  - `originator_iso`: STRING Example: "SGP"\n  - `beneficiary_bank_id`: STRING Example: "barclays-bank-plc-london-england-gbr"\n  - `origin_lat`: STRING Example: "1.3667"\n  - `origin_lng`: STRING Example: "103.8"\n  - `end_date_format`: STRING Example: "2015-09-25T00:00:00Z"\n  - `begin_date_format`: STRING Example: "2015-03-25T00:00:00Z"\n  - `originator_bank`: STRING Example: "CIMB Bank Berhad"\n  - `beneficiary_lat`: STRING Example: "54"\n  - `beneficiary_iso`: STRING Example: "GBR"\n  - `beneficiary_lng`: STRING Example: "-2"\n  - `begin_date`: STRING Example: "Mar 25, 2015"\n  - `id`: STRING Example: "223254"\n  - `end_date`: STRING Example: "Sep 25, 2015"\n  - `amount`: INTEGER Min: 1.18, Max: 2721000000\n  - `number`: INTEGER Min: 1, Max: 174\n- **Entity**\n  - `id`: STRING Example: "the-bank-of-new-york-mellon-corp"\n  - `location`: POINT \n  - `name`: STRING Example: "The Bank of New York Mellon Corp."\n  - `country`: STRING Example: "CHN"\nRelationship properties:\n\nThe relationships:\n(:Filing)-[:BENEFITS]->(:Entity)\n(:Filing)-[:CONCERNS]->(:Entity)\n(:Filing)-[:ORIGINATOR]->(:Entity)\n(:Entity)-[:FILED]->(:Filing)\n(:Entity)-[:COUNTRY]->(:Country) \n Question: Which 3 countries have the most entities linked as beneficiaries in filings?  \n Cypher output: '}].<|im_end|>
<|im_start|>assistant
Alright, so I need to figure out which three countries have the most entities linked as beneficiaries in filings using the provided schema and relationships. Let me break this down step by step.

First, I'll look at the schema provided. There are two main node types: **Country** and **Filing**. The **Filing** node has several properties, including `originator_bank_country` and `beneficiary_bank_country`, both of which are strings representing countries. Additionally, there's a relationship property `:COUNTRY` that links **Entity** nodes to **Country** nodes. However, the question is about entities linked as beneficiaries, so I might not need the `:COUNTRY` relationship directly unless I'm considering the country context of the entities.

The relationships given are:
- (:Filing)-[:BENEFITS]->(:Entity)
- (:Filing)-[:CONCERNS]->(:Entity)
- (:Filing)-[:ORIGINATOR]->(:Entity)
- (:Entity)-[:FILED]->(:Filing)
- (:Entity)-[:COUNTRY]->(:Country)

But since the question is about entities linked as beneficiaries, I should focus on the `:BENEFITS` relationship. Each `:Filing` node can have multiple `:Entity` nodes connected via `:BENEFITS`, meaning each entity is a beneficiary in a filing.

To find the countries with the most entities as beneficiaries, I need to:
1. Identify all `:Entity` nodes connected via `:BENEFITS` to `:Filing` nodes.
2. For each of these entities, find their `beneficiary_bank_country` property, which should be the country of the bank acting as the beneficiary.
3. Count how many entities have their `beneficiary_bank_country` as each country.
4. Sort these countries by the count of entities and select the top three.

I should also consider if there are any constraints or properties that might affect this count. For example, if an entity appears multiple times in different filings, each occurrence should be counted separately. However, the schema doesn't specify if an entity can be linked multiple times to the same country, so I'll assume each connection is a separate count.

Now, translating this into a Cypher query. I need to:
- Start with the `:Entity` nodes.
- Filter those that have a `:BENEFITS` relationship to a `:Filing`.
- Group these entities by their `beneficiary_bank_country`.
- Count the number of entities per country.
- Order the results by count in descending order.
- Select the top three countries.

Putting this into a Cypher statement, it would look something like this:

```cypher
MATCH (e:Entity {relationship: [:BENEFITS]})
 OPTIONAL MATCH (f:Filing)-[:BENEFITS]->(e)
 RETURN e.beneficiary_bank_country, COUNT(e) AS count
ORDER BY DESC(count)
LIMIT 3
```

Wait, that might not be exactly right. Let me refine it. I need to ensure that I'm only considering entities that have at least one `:BENEFITS` relationship. Also, the `relationship` property in the `:BENEFITS` relationship is optional, so I should adjust the match pattern accordingly.

Perhaps a better approach is:

```cypher
MATCH (:Filing)-[:BENEFITS]->(:Entity)
 OPTIONAL MATCH (:Filing)-[:BENEFITS]->(:Entity)
 OPTIONAL MATCH (:Filing)-[:BENEFITS]->(:Entity)
  ... but this seems repetitive. Instead, I can use a subquery to get all entities connected via `:BENEFITS` to `:Filing`.

Wait, maybe a better way is to use a projection that includes all entities that are connected via `:BENEFITS` to `:Filing`. Then, group by `beneficiary_bank_country` and count.

So, the correct Cypher statement would be:

```cypher
MATCH (:Filing)-[:BENEFITS]->(:Entity) AS e
GROUP BY e.beneficiary_bank_country
ORDER BY COUNT(e) DESC
LIMIT 3
```
# MATCH (f:Filing)-[:BENEFITS]->(e:Entity)-[:COUNTRY]->(c:Country)
# WITH c.name AS country, COUNT(e) AS entityCount
# ORDER BY entityCount DESC
# LIMIT 3
# RETURN country, entityCount

But I think I need to ensure that each entity is only counted once per country, even if they appear in multiple filings. However, the question asks for the number of entities linked as beneficiaries, so each unique entity per country is counted once, regardless of how many times they appear in different filings.

Wait, no. If an entity is a beneficiary in multiple filings, it should still be counted once per country, but the count should reflect how many times the entity is linked to that country. But I think the question is asking for the number of unique entities per country, not the number of times they appear in filings.

Wait

'<｜begin▁of▁sentence｜><|im_start|>[{\'role\': \'user\', \'content\': \'Generate Cypher statement to query a graph database. Use only the provided relationship types and properties in the schema. \\nSchema: Node properties:\\n- **Country**\\n  - `location`: POINT \\n  - `code`: STRING Example: "AFG"\\n  - `name`: STRING Example: "Afghanistan"\\n  - `tld`: STRING Example: "AF"\\n- **Filing**\\n  - `begin`: DATE_TIME Min: 2000-02-08T00:00:00Z, Max: 2017-09-05T00:00:00Z\\n  - `end`: DATE_TIME Min: 2000-02-08T00:00:00Z, Max: 2017-11-03T00:00:00Z\\n  - `originator_bank_id`: STRING Example: "cimb-bank-berhad"\\n  - `sar_id`: STRING Example: "3297"\\n  - `beneficiary_bank`: STRING Example: "Barclays Bank Plc"\\n  - `filer_org_name_id`: STRING Example: "the-bank-of-new-york-mellon-corp"\\n  - `originator_bank_country`: STRING Example: "Singapore"\\n  - `beneficiary_bank_country`: STRING Example: "United Kingdom"\\n  - `filer_org_name`: STRING Example: "The Bank of New York Mellon Corp."\\n  - `

In [ ]:

tokenizer.pad_token = tokenizer.eos_token
question = "What are the movies of Tom Hanks?"
schema = "(:Actor)-[:ActedIn]->(:Movie)"
new_message = prepare_chat_prompt(question=question, schema=schema)
prompt = f"<|im_start|>{new_message}.<|im_end|>\n<|im_start|>assistant\n"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=1000,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)
response = tokenizer.decode(outputs[0], skip_special_tokens=False)
response



'<｜begin▁of▁sentence｜><|im_start|>[{\'role\': \'user\', \'content\': \'Generate Cypher statement to query a graph database. Use only the provided relationship types and properties in the schema. \\nSchema: (:Actor)-[:ActedIn]->(:Movie) \\n Question: What are the movies of Tom Hanks?  \\n Cypher output: \'}].<|im_end|>\n<|im_start|>assistant\nOkay, so I need to generate a Cypher statement to query a graph database. The user provided a schema and a question, so let me break this down step by step. \n\nFirst, the schema is (:Actor)-[:ActedIn]->(:Movie). This means that there\'s a relationship between Actors and Movies where the actor has acted in the movie. So, the relationship type is ActedIn, connecting an Actor to a Movie.\n\nThe question is: "What are the movies of Tom Hanks?" So, essentially, I need to find all the Movies that are connected to Tom Hanks through the ActedIn relationship.\n\nIn Cypher, to query this, I would start by selecting the Movie nodes. Then, I need to find the 

In [ ]:
<｜begin▁of▁sentence｜><|im_start|>[{'role': 'user', 'content': 'Generate Cypher statement to query a graph database. Use only the provided relationship types and properties in the schema. \nSchema: (:Actor)-[:ActedIn]->(:Movie) \n Question: What are the movies of Tom Hanks?  \n Cypher output: '}].<|im_end|>
<|im_start|>assistant
Okay, so I need to generate a Cypher statement to query a graph database. The user provided a schema and a question, so let me break this down step by step.

First, the schema is (:Actor)-[:ActedIn]->(:Movie). This means that there's a relationship between Actors and Movies where the actor has acted in the movie. So, the relationship type is ActedIn, connecting an Actor to a Movie.

The question is: "What are the movies of Tom Hanks?" So, essentially, I need to find all the Movies that are connected to Tom Hanks through the ActedIn relationship.

In Cypher, to query this, I would start by selecting the Movie nodes. Then, I need to find the connections between Tom Hanks (as an Actor) and the Movies. So, the FROM clause would start with Movie, but I think it's more straightforward to specify the relationship type directly in the WHERE or the projection.

Wait, actually, in Cypher, you can use the relationship directly in the projection. So, I can write something likeMATCH (Actor {name: 'Tom Hanks'})-[:ActedIn]->(Movie). Then, to collect all the Movies, I can use the WITH clause to project the Movie properties.

Putting it all together, the Cypher statement would match the path where Tom Hanks is connected via ActedIn to a Movie, and then return those Movies. So, the statement would be:

MATCH (Actor {name: 'Tom Hanks'})-[:ActedIn]->(Movie)
WITH Movie
RETURN Movie
# ["MATCH (a:Actor {Name: 'Tom Hanks'})-[:ActedIn]->(m:Movie) RETURN m"]

This should return all the Movies that Tom Hanks has acted in. I think that covers the user's request. I should make sure to use the correct syntax, like single quotes around the property values and proper case for relationship types.

I don't think I need to include any additional clauses unless the user specified something else, but for this question, this should suffice. Also, I should ensure that the Cypher is case-sensitive, so the relationship type ActedIn should be in uppercase as per the schema.

So, the final output would be the Cypher statement as above, without any extra information or explanations, just the code block.
</think>

<|im_start|>[{'role': 'assistant', 'content': 'MATCH (Actor {name: \"Tom Hanks\"})-[:ActedIn]->(Movie)\n WITH Movie\n RETURN Movie'}].<|im_end|><｜end▁of▁sentence｜>

In [ ]:
torch.cuda.empty_cache()